In [1]:
import json
import requests
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize 
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [9]:
# match_id 4986461644 is game 4 ti9 grand finals
url = 'https://api.opendota.com/api/matches/4986461644'
r = requests.get(url)
data = r.json()

In [10]:
df = pd.json_normalize(data)
df.head()

,match_id,barracks_status_dire,barracks_status_radiant,chat,cluster,dire_score,dire_team_id,draft_timings,duration,engine,first_blood_time,game_mode,human_players,leagueid,lobby_type,match_seq_num,negative_votes,objectives,picks_bans,positive_votes,radiant_gold_adv,radiant_score,radiant_team_id,radiant_win,radiant_xp_adv,skill,start_time,teamfights,tower_status_dire,tower_status_radiant,version,replay_salt,series_id,series_type,players,patch,throw,loss,replay_url,cosmetics.647,cosmetics.4269,cosmetics.4538,cosmetics.4560,cosmetics.4580,cosmetics.5390,cosmetics.5406,cosmetics.5732,cosmetics.5967,cosmetics.6005,cosmetics.6432,cosmetics.6433,cosmetics.6434,cosmetics.6526,cosmetics.6687,cosmetics.6758,cosmetics.7580,cosmetics.7668,cosmetics.7935,cosmetics.7994,cosmetics.8203,cosmetics.8263,cosmetics.8964,cosmetics.9235,cosmetics.9526,cosmetics.9527,cosmetics.9528,cosmetics.9529,cosmetics.9530,cosmetics.9748,cosmetics.9762,cosmetics.11266,cosmetics.11321,cosmetics.12920,cosmetics.12947,cosmetics.12996,cosmetics.13235,cosmetics.13236,cosmetics.13262,cosmetics.13263,cosmetics.13264,cosmetics.13523,cosmetics.13541,cosmetics.13563,cosmetics.13564,cosmetics.13565,cosmetics.13566,cosmetics.16286,cosmetics.16674,cosmetics.17482,league.leagueid,league.ticket,league.banner,league.tier,league.name,radiant_team.team_id,radiant_team.name,radiant_team.tag,radiant_team.logo_url,dire_team.team_id,dire_team.name,dire_team.tag,dire_team.logo_url,all_word_counts.glhf,all_word_counts.gl,all_word_counts.hf,all_word_counts.gg,all_word_counts.ggg
0,4986461644,51,63,"[{'time': -100, 'type': 'chatwheel', 'key': '2...",308,11,2163,"[{'order': 1, 'pick': False, 'active_team': 2,...",1474,1,121,2,10,10749,1,4182489532,445,"[{'time': 121, 'type': 'CHAT_MESSAGE_FIRSTBLOO...","[{'is_pick': False, 'hero_id': 52, 'team': 0, ...",8963,"[0, 185, 193, 451, 40, 42, -130, -604, -1114, ...",20,2586976,True,"[0, -131, 5, -86, -27, 129, 963, 256, -290, -4...",None,1566733817,"[{'start': 1212, 'end': 1261, 'last_death': 12...",1926,1972,21,1194860475,360940,2,"[{'match_id': 4986461644, 'player_slot': 0, 'a...",41,1577,7103,http://replay308.valve.net/570/4986461644_1194...,132,129,129,129,129,132,128,130,4,131,131,131,131,131,132,132,129,132,131,128,4,132,132,1,3,3,3,3,3,129,129,129,3,2,0,3,0,0,0,0,0,3,2,2,2,2,2,2,132,2,10749,None,None,premium,The International 2019,2586976,OG,OG,https://steamcdn-a.akamaihd.net/apps/dota2/ima...,2163,Team Liquid,Liquid,https://steamcdn-a.akamaihd.net/apps/dota2/ima...,2,2,3,6,1


## Normalize 'picks_bans'

Based on research, want to format 'picks_bans' object into single row such that each column we want turns into 22 columns. 

Reference: https://github.com/codexponent/dota2-draft-analysis/blob/master/README.md
Reference: https://medium.com/@shrinivasshetty21/deep-learning-with-keras-on-dota-2-statistics-9a937434f9e0

### ALTERNATIVE METHOD 

Source: https://www.kaggle.com/davidmercury/hero-picking-and-match-result

In [17]:
df = df[['match_id', 'radiant_win']]

In [14]:
picks_df = json_normalize(df['picks_bans'][0])
#df_picks.drop(index=None, columns=['ord'], inplace=True)
picks_df = picks_df[['match_id', 'order', 'is_pick', 'team', 'hero_id']]
picks_df

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,match_id,order,is_pick,team,hero_id
0,4986461644,0,False,0,52
1,4986461644,1,False,1,97
2,4986461644,2,False,0,55
3,4986461644,3,False,1,73
4,4986461644,4,False,0,58
5,4986461644,5,False,1,103
6,4986461644,6,True,0,91
7,4986461644,7,True,1,66
8,4986461644,8,True,1,57
9,4986461644,9,True,0,19


In [18]:
picks_df = pd.merge(df, picks_df, on = 'match_id',how = 'left')
picks_df.set_index(['match_id','order'],inplace = True)
picks_df

radiant_win  is_pick  team  hero_id
match_id   order                                     
4986461644 0             True    False     0       52
           1             True    False     1       97
           2             True    False     0       55
           3             True    False     1       73
           4             True    False     0       58
           5             True    False     1      103
           6             True     True     0       91
           7             True     True     1       66
           8             True     True     1       57
           9             True     True     0       19
           10            True    False     0       82
           11            True    False     1       95
           12            True    False     0      106
           13            True    False     1       74
           14            True     True     1       99
           15            True     True     0      102
           16            True     True     1      107
           17            True     True     0       98
           18            True    False     1       53
           19            True    False     0       11
           20            True     True     0       72
           21            True     True     1       21

In [21]:
id_df = picks_df[['is_pick', 'team', 'hero_id']].unstack()
id_df.head()

is_pick                                                             \
order           0      1      2      3      4      5     6     7     8     9    
match_id                                                                        
4986461644   False  False  False  False  False  False  True  True  True  True   

                                                                              \
order          10     11     12     13    14    15    16    17     18     19   
match_id                                                                       
4986461644  False  False  False  False  True  True  True  True  False  False   

                       team                                                  \
order         20    21   0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16   
match_id                                                                      
4986461644  True  True    0  1  0  1  0  1  0  1  1  0  0  1  0  1  1  0  1   

                          hero_id                                           \
order      17 18 19 20 21      0   1   2   3   4    5   6   7   8   9   10   
match_id                                                                     
4986461644  0  1  0  0  1      52  97  55  73  58  103  91  66  57  19  82   

                                                           
order       11   12  13  14   15   16  17  18  19  20  21  
match_id                                                   
4986461644  95  106  74  99  102  107  98  53  11  72  21

Convert data frame to string, aggregate the desired columns into a single row s.t. all values in column expressed as a string 

In [1]:
df = df.astype(str)
df = df.groupby('match_id')['is_pick','team','hero_id'].agg(' '.join)
df

NameError: name 'df' is not defined

In [14]:
#new = df['order'].str.split(' ', expand=True)
new1 = df['is_pick'].str.split(' ', expand=True)
new2 = df['team'].str.split(' ', expand=True)
new3 = df['hero_id'].str.split(' ', expand=True)
new1, new2, new3

(                0      1      2      3      4      5     6     7     8     9  \
 match_id                                                                       
 4986461644  False  False  False  False  False  False  True  True  True  True   
 
                10     11     12     13    14    15    16    17     18     19  \
 match_id                                                                       
 4986461644  False  False  False  False  True  True  True  True  False  False   
 
               20    21  
 match_id                
 4986461644  True  True  ,
             0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21
 match_id                                                                    
 4986461644  0  1  0  1  0  1  0  1  1  0  0  1  0  1  1  0  1  0  1  0  0  1,
              0   1   2   3   4    5   6   7   8   9  10  11   12  13  14   15  \
 match_id                                                                        
 4986461644  52  97  55  73  58 